In [ ]:
! pip install PyPDF2
import numpy as np
import pandas as pd
import fpdf
import os
from fpdf import FPDF
import csv
from PyPDF2 import PdfFileMerger
import time

In [ ]:
def createList(r1, r2):
    return list(range(r1, r2+1))

In [ ]:
def pdf_file_merger(path,ISO):
    #Create and instance of PdfFileMerger() class
    merger = PdfFileMerger()
    #Create a list with PDF file names
    path_to_files = r''+path+'/'
    #Get the file names in the directory
    for root, dirs, file_names in os.walk(path_to_files):
        #Iterate over the list of file names
        for file_name in file_names:
            #Append PDF files
            merger.append(path_to_files + file_name)
    #Write out the merged PDF
    merger.write(path+'/'+ISO+".pdf")
    merger.close()

In [ ]:
class PDF(FPDF):
    def setHeading(self, heading):
        self.heading = heading
    def header(self):
        self.l_margin = 14
        self.r_margin = 14
        
        self.set_font('Arial', '', hdg_font_sz)
        #self.set_text_color(43,60,102)
        self.set_text_color(0,0,0)
        self.cell(0, 10, f'{self.heading}', 0, 0, 'C')

        #self.set_font('Arial', '', rpt_font_sz)
        #self.cell(-30, 11, f'{date_fmt}', 0, 0, 'C')
        self.ln(15)

        self.set_fill_color(240,248,255)

        col = 0
        while col < len(rpt_hdgs):
            col_width = cwidths[col]
            self.cell(col_width, line_height, rpt_hdgs[col], 0, 0, fill=True)    
            col += 1   
        self.ln(15)
    
    def footer(self):
        # Go to 1.5 cm from bottom
        self.set_y(-15)
        # Select Arial italic 8
        self.set_font('Arial', 'I', 10)
        # Print centered page number
        self.cell(0, 10, 'Page %s' % self.page_no(), 0, 0, 'C')

In [ ]:
def convert_to_pdf(data_list,pdf_name,heading,path):
    #fpdf.set_global("SYSTEM_TTFONTS", os.path.join(os.path.dirname('/Users/msdhoni/Automation/Repos/test-automation'),'fonts'))
    pdf = PDF()
    pdf.setHeading(heading)
    pdf.alias_nb_pages()
    #pdf.set_auto_page_break(auto: bool, margin = 1.0)
    pdf.add_page()
    #pdf.set_font('Arial', '', rpt_font_sz, uni=True)
    pdf.add_font("NotoSans", style="", fname="NotoSans-Regular.ttf", uni=True)
    pdf.set_font('NotoSans', '', rpt_font_sz)
    #pdf.add_font("NotoSans", style="B", fname="NotoSans-Bold.ttf", uni=True)
    #pdf.add_font("NotoSans", style="I", fname="NotoSans-Italic.ttf", uni=True)
    #pdf.add_font("NotoSans", style="BI", fname="NotoSans-BoldItalic.ttf", uni=True)
    row_count = 0
    while row_count < len(data_list):
        col = 0
        for c in cwidths:
            pdf.cell(c, 0, data_list[row_count][col], align='L', border=0)
            col += 1
        pdf.ln(7)
        row_count += 1
    pdf.output(path+'/'+pdf_name, 'F')

In [ ]:
def get_side_heading(adm1_unique_values,index,file_name1):
    adm1_list = adm1_unique_values
    count = index
    side_heading =  adm1_list[count]+", "+file_name1
    return side_heading

In [ ]:
def convert_to_pdf1(data_list,pdf_name,heading,path,adm1_unique_values,file_name1):
    pdf = PDF()
    pdf.setHeading(heading)
    pdf.alias_nb_pages()
    pdf.set_auto_page_break(auto = True, margin = 14)
    pdf.add_page()
    pdf.add_font("NotoSans", style="", fname="NotoSans-Regular.ttf", uni=True)
    pdf.set_font('NotoSans', '', rpt_font_sz)
    row_count = 0
    count = 0
    adm1_list = adm1_unique_values
    while row_count < len(data_list):
        col = 0
        if ((int(data_list[row_count][col])) == 1):
            side_heading = get_side_heading(adm1_unique_values,count,file_name1)
            pdf.ln(4)
            pdf.cell(0, 0, f'{side_heading}', align='L', border=0)
            pdf.ln(7)
            count += 1
        for c in cwidths:
            pdf.cell(c, 0, data_list[row_count][col], align='L', border=0)
            col += 1
        pdf.ln(7)
        row_count += 1
    pdf.output(path+'/'+pdf_name, 'F')
    #pdf.output("AND", 'F')

In [ ]:
data = pd.read_csv('AND.csv')

In [ ]:
data

In [ ]:
data.loc[:, "total_p"] = data["total_p"].map('{:,d}'.format)

In [ ]:
file_name1 = data['adm0_nm'][0]
adm1_values = data['adm1_nm'].values
adm1_unique_values = list(np.unique(adm1_values))
directory = "pandu"
temp = directory
parent_dir = "/Users/msdhoni/Automation/Repos/pdfs"
path = os.path.join(parent_dir, directory)
os.mkdir(path)
data_list_final = []
for item in adm1_unique_values:
    needed_data = data.query('adm1_nm == "'+item+'"')
    needed_data = needed_data[['adm4_nm','total_p']]
    count_row = needed_data.shape[0]
    index_list = createList(1, count_row)
    index = pd.Index(index_list)
    needed_data = needed_data.set_index(index)
    needed_data.to_csv('example.csv')
    with open(f'example.csv', 'r') as csvfile:
        data_list= list(csv.reader(csvfile))[1:]
    data_list_final.append(data_list)
    #list(data_list_final,"utf-8")
    #data_list_final.decode("utf-8")
    os.remove('example.csv')
final_list = []
for temp in data_list_final:
    for elem in temp:
        final_list.append(elem)
#print(data_list[:3])
#pdf_name = f'example_and.pdf'
file_name2 = item
#file_name2 = file_name2.replace(" ","")
pdf_name = file_name1 +'_'+ file_name2 + '.pdf'
heading = file_name1
rpt_hdgs = [' ', 'Village, City or Settlement', 'total population'] 
cwidths  = [10, 90, 55]
rpt_font_sz = 12
hdg_font_sz = 15
line_height = 8
#convert_to_pdf1(data_list_final,pdf_name,heading,path)
convert_to_pdf1(final_list,pdf_name,heading,path,adm1_unique_values,file_name1)
#os.remove('example.csv')
#pdf_file_merger(path,"AND")"""

In [ ]:
print(len(data_list_final[1]))
row_count1 = 0
while row_count1 < len(data_list_final):
    row_count2 = 0
    while row_count2 < len(data_list_final[row_count1]):
        col = 0
        for c in cwidths:
            #pdf.cell(c, 0, data_list_final[row_count][col], align='L', border=0)
            print(data_list_final[row_count1][col])
            col += 1
            #pdf.ln(7)
        row_count2 += 1
    row_count1 += 1